In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
#more imports; mine, not there by default on Kaggle Kernel
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer
mae_scoring = make_scorer(mean_absolute_error)

from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsRegressor

from numbers import Number

In [10]:
df = pd.read_csv("./input/train.csv")
y = df['loss']

#write losses to csv
y.to_csv("./transf/loss.csv")

In [6]:
#find columns w nas
print(df.columns[pd.isnull(df).any()].tolist())
#none

#get source (non-id, non-loss) column names
src = list(df.columns)
src.remove('id')
src.remove('loss')

len(src)

[]


130

In [8]:
#dummy out categorical variables so we can use numerical dimensionality reduction + other methods
ind = 1
for var in src:
    if var[:3] == "cat":
#         print(var)
        dums = pd.get_dummies(df[var], prefix = "dum" + str(ind), drop_first = True )
        df = pd.concat([df,dums], axis=1)
        ind += 1

In [19]:
#write dataframe with all necessary columns to csv
# df.to_csv("./transf/train_dummies.csv")

In [16]:
#write dataframe w dummies and conts only to csv

#inputs = [col for col in df.columns if(col[:3] == 'dum' or col[:4] == 'cont')]
#df = df[inputs]

# df.to_csv("./transf/train_dummies_nocats.csv")

In [7]:
#normalize continuous variables
cols_to_norm = [col for col in df.columns if col[:4] == "cont"]

df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.mean()) / np.sqrt(x.var()))

In [30]:
#write df with dummies and normalized conts to csv
#df.to_csv("./transf/train_inputs.csv")

#read df w dummies and normalized conts - using a fraction of rows so I can try things out without being too slow
df = pd.read_csv("./transf/train_inputs.csv", nrows = 10000)

cont1        0.990890
cont2        0.996037
cont3        1.009693
cont4        0.998074
cont5        0.973569
cont6        0.989288
cont7        1.023593
cont8        1.006121
cont9        0.997499
cont10       0.987978
cont11       1.012633
cont12       1.010119
cont13       0.996892
cont14       1.004844
dum1_B       0.185350
dum2_B       0.245068
dum3_B       0.051178
dum4_B       0.216642
dum5_B       0.228502
dum6_B       0.213119
dum7_B       0.024377
dum8_B       0.055701
dum9_B       0.239518
dum10_B      0.126249
dum11_B      0.094142
dum12_B      0.126741
dum13_B      0.091446
dum14_B      0.011760
dum15_B      0.000100
dum16_B      0.031073
               ...   
dum116_ME    0.001598
dum116_MF    0.000000
dum116_MG    0.001797
dum116_MH    0.000000
dum116_MI    0.000600
dum116_MJ    0.003289
dum116_MK    0.000200
dum116_ML    0.000100
dum116_MM    0.000000
dum116_MN    0.000100
dum116_MO    0.000300
dum116_MP    0.000400
dum116_MQ    0.000000
dum116_MR    0.000100
dum116_MS 

In [31]:
df = np.asarray(df[inputs])

pca = PCA()

pca.fit_transform(df)

exp = pca.explained_variance_ratio_
cumvar = np.cumsum(exp)
plt.clf()
plt.plot(range(len(cumvar)), cumvar)
plt.show()
#99% of variance is included in first 353 components

In [36]:
n_take = 353
X = df[:,:n_take]

yold = list(y)
y = yold[:len(X)]

In [37]:
#testing out some regressors, getting a sense of what error they have
#even after trimming down features with PCA, fitting a regressor takes a while: 10+min for an mae-criterion decision tree
#seems like a good potential place for a booster - might be many subpatterns to fit local greedy algs to

#tried linear regression just to see - it performs abysmally (error ~ 10^6)
#Decision Tree opt for mse gets ~1700
#Random Forest opt for mse gets ~1400

reg = DecisionTreeRegressor(criterion = 'mae')
# reg = RandomForestRegressor(n_estimators = 3, criterion = 'mae')

score = cross_val_score(reg,X,y,scoring = mae_scoring)
score

array([ 1860.00926815,  1856.41639364,  1886.26390639])

In [ ]:
reg = kNeighborsRegressor(weight = 'uniform')
tovary = "n_neighbors"
vary_range = range(2,6,2)

train_scores, valid_scores = learning_curve(reg, X,y, tovary, vary_range)

trainavg = np.mean(train_scores, axis=1)
validavg = np.mean(valid_scores, axis=1)

plt.clf()
plt.plot(vary_range, trainavg, color = 'k', marker = '.')
plt.plot(vary_range, validavg, color = 'g', marker = '.')


In [ ]:
#clean test data

In [1]:
dft = pd.read_csv("./input/train.csv")

NameError: name 'pd' is not defined

In [ ]:
#find columns w nas
print(dft.columns[pd.isnull(dft).any()].tolist())
#none

#get source (non-id, non-loss) column names
src = list(dft.columns)
src.remove('id')
src.remove('loss')

len(src)

NameError: name 'dft' is not defined

In [8]:
#dummy out categorical variables so we can use numerical dimensionality reduction + other methods
ind = 1
for var in src:
    if var[:3] == "cat":
#         print(var)
        dums = pd.get_dummies(dft[var], prefix = "dum" + str(ind), drop_first = True )
        dft = pd.concat([dft,dums], axis=1)
        ind += 1

In [19]:
#write dataframe with all columns to csv
dft.to_csv("./transf/test_dummies.csv")

In [16]:
#write dataframe w dummies and conts only to csv

dft = dft[inputs]

dft.to_csv("./transf/test_dummies_nocats.csv")

In [7]:
#normalize continuous variables
cols_to_norm = [col for col in dft.columns if col[:4] == "cont"]

dft[cols_to_norm] = dft[cols_to_norm].apply(lambda x: (x - x.mean()) / np.sqrt(x.var()))

#write df with dummies and normalized conts to csv
dft.to_csv("./transf/test_inputs.csv")

##read df w dummies and normalized conts - using a fraction of rows so I can try things out without being too slow
#dft = pd.read_csv("./transf/train_inputs.csv")